## Extração de dados sobre o coronavírus

### Dados da Secretaria de Saúde de Pernambuco


In [32]:
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd
import tabula
from datetime import datetime


### Pegando informações da página

Primeiro vamos pegar o código da página da Secretaria de Saúde

No caso da Secretaria de Pernambuco, os links de interesse ficam na
tag **h1**. Por isso iremos buscar por elas no documento.

In [33]:
url = 'https://www.cievspe.com/novo-coronavirus-2019-ncov'

html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')


h1 = soup.find_all('h1', 'font_0')

### Fazendo download do arquivo

Após criar uma lista com todos os H1s do site, iteramos procurando 
o conteúdo da tag **span**, pois nela estão os nomes dos arquivos que
queremos baixar.

Ao encontrar o arquivo **Informe Epidemiológico Coronavírus (COVID-19)**, 
que possui uma numeração com a atualização, fazemos o dowload do 
PDF que será tratado posteriormente.

In [34]:
date = ""

for span in h1:
    if span.find('span') is not None:
        if 'Informe Epidemiológico Coronavírus (COVID-19)' in span.text:
            print(span.text)
            data = span.text.split(" ") 
            date = datetime.strptime(data[0], '%d/%m/%Y').date()
            print(date)
            print(span.find('a')['href'])
            
            filename = Path('covid19_pe.pdf')
            url = span.find('a')['href']
            response = requests.get(url)
            filename.write_bytes(response.content)


20/03/2020 Informe Epidemiológico Coronavírus (COVID-19) | Nº 19 - Pernambuco
2020-03-20
https://12ad4c92-89c7-4218-9e11-0ee136fa4b92.filesusr.com/ugd/3293a8_bb922dee19844ac7b24fe5649aa692a0.pdf


 Abaixo está a conversão da tabela em PDF para o formato CSV.
 
 * Tabula é uma biblioteca que lê a tabela magicamente 
 * 4 é a página que tem a tabela de interesse
 

In [67]:
tabula.convert_into("covid19_pe.pdf", "covid19_pe.csv", output_format="csv", pages=4)

df = pd.read_csv('covid19_pe.csv')

### Tratando os dados da tabela

Após criar o CSV com os dados, precisamos trocar os nomes das colunas
de números (1 a 6) para algo mais explícito. Após isso, salvamos as
informações no CSV novamente.

In [68]:
df = df.T.reset_index(drop=True).T
df = df.drop([0,1])
df = df.iloc[ :,1:]

df = df.rename(columns={1:'MUNICIPIO', 2:'SUSPEITO', 3:'PROVAVEL', 4:'CONFIRMADO', 5:'DESCARTADO', 6:'TOTAL'})

cols = ['SUSPEITO', 'PROVAVEL', 'CONFIRMADO', 'DESCARTADO', 'TOTAL']
for col in cols:
    df[col] = df[col].map(lambda x: str(x).lstrip('*').rstrip('*'))
    
df['MUNICIPIO'] = df['MUNICIPIO'].map(lambda x: str(x).replace('\n', ' '))
df = df.sort_values(['MUNICIPIO'], ascending=1)
df = df.reset_index(drop=True)
df['DATETIME'] = date


In [69]:
df.drop( df[df['MUNICIPIO'] == 'Subtotal'].index, axis=0, inplace=True)

df.dropna(thresh=2, inplace=True, axis=0)

df.head(50)

df.to_csv('covid19_pe.csv')
